# se.plan calculating areas adjusted by restoration potential


1. Load a previous se.plan suitability raster
2. Add a new AOI that would be larger than typically usable in the se.plan GUI
3. Calculate the adjusted area 
4. Export the new recipe to a GEE asset and export our statistics to Google Drive as a CSV



### Step 1 - Load a previous se.plan data



In [3]:
# import libaraies needed  and initialize to GEE.
# from component import scripts
import ee
import json
import pandas as pd
#import geemap
ee.Initialize()

In [2]:
#path = '/home/amahon/module_results/se.plan/KHM/recipe_2022-09-12_equalweight.json'
#recipe_name = 'equalweight'
#with open(path, 'r') as f:
#  recipe = json.load(f)

# get priorities and constraints AS STRINGS
# wlc function expects this
#priorities = recipe['question_model']['priorities']
#constraints = recipe['question_model']['constraints']
#layer_list = recipe['layer_model']['layer_list']

#print('wlc priorities input type should be a string :',type(priorities))
#print('wlc constraints input type should be a string :',type(constraints))
#print('layer_list should be a list :',type(layer_list))

In [4]:
wlc_asset = ee.Image("projects/ee-amcmahon/assets/seplan-test_recipe_2022-09-12_equalweight")

In [5]:
#add layer with tree fraction
tree_fraction = ee.Image('projects/john-ee-282116/assets/fao-restoration/staging/restoration_potential_gt_treecoverfrac_mask_urban_diff')

### Step 2 - Add a new AOI


In [6]:
aoi = ee.FeatureCollection('projects/ee-amcmahon/assets/seplan/RAP_aoi_union')

fao = ee.FeatureCollection("FAO/GAUL/2015/level0")
country_list = ee.List(['Afghanistan', 'Bangladesh', 'Bhutan', 'Cambodia', 'Fiji', 'India', 'Indonesia', 'Malaysia', 'Mongolia', 'Myanmar', 'Nepal', "Lao People's Democratic Republic", 'Pakistan', 'Papua New Guinea','Philippines', 'Solomon Islands', 'Sri Lanka', 'Thailand', 'Timor-Leste', 'Vanuatu', 'Viet Nam'])
#removed countries: 
#'Samoa' - no aboveground biomass layer

sub_aois = fao.filter(ee.Filter.inList("ADM0_NAME", country_list))

In [7]:
# jjd test
wlc_adj = wlc_asset.multiply(tree_fraction)
wlc = wlc_adj, [], [], []
dashboard_data = scripts.get_summary_statistics(wlc, 'mekong-test', aoi, [], False)

(<ee.image.Image at 0x2557f493b80>, [], [], [])

### Step 3: calculate area adjusted by tree fraction

In [13]:
def get_sub_aois_adjusted_areas(sub_aois:ee.FeatureCollection, ids:list, ids_col:str, wlc_image:ee.Image):

    def get_sub_stats(sub_id):
        geom = sub_aois.filter(ee.Filter.eq(ids_col,sub_id)).first().geometry()
        image = wlc_image.clip(geom)
        image = image.rename("image").round()

        pixelArea = ee.Image.pixelArea().divide(10000)
        pixelArea = pixelArea.multiply(tree_fraction)

        reducer = ee.Reducer.sum().group(1, "image")

        areas = (
            pixelArea.addBands(image)
            .reduceRegion(reducer=reducer, geometry=sub_aois, scale=100, maxPixels=1e12)
            .get("groups")
        )

        areas_list = ee.List(areas).map(lambda i: ee.Dictionary(i).get("sum"))

        featureCollection = ee.FeatureCollection(areas_list)

        
        return (sub_id, featureCollection)
    
    return [get_sub_stats(sub_id) for sub_id in ids]

In [14]:
# print(sub_aois.first().toDictionary().getInfo())
sub_id_col = 'ADM0_NAME'
sub_ids = sub_aois.aggregate_array(sub_id_col).getInfo()

adjusted_area_data_sub_aois = get_sub_aois_adjusted_areas(sub_aois= sub_aois, ids=sub_ids, ids_col=sub_id_col, wlc_image=wlc_asset)
#adjusted_area_data_sub_aois.append( ('RAP_aoi_union',dashboard_data) )
adjusted_area_data_sub_aois

[('Mongolia', <ee.featurecollection.FeatureCollection at 0x7f51beb79e80>),
 ('Myanmar', <ee.featurecollection.FeatureCollection at 0x7f51beb7f6d0>),
 ('Cambodia', <ee.featurecollection.FeatureCollection at 0x7f51beb771f0>),
 ('Viet Nam', <ee.featurecollection.FeatureCollection at 0x7f51beb79730>),
 ("Lao People's Democratic Republic",
  <ee.featurecollection.FeatureCollection at 0x7f51beb9f040>),
 ('Malaysia', <ee.featurecollection.FeatureCollection at 0x7f51beb9fd00>),
 ('Thailand', <ee.featurecollection.FeatureCollection at 0x7f51beb61a00>),
 ('Philippines', <ee.featurecollection.FeatureCollection at 0x7f51be9b3700>),
 ('Indonesia', <ee.featurecollection.FeatureCollection at 0x7f51be9d5400>),
 ('Timor-Leste', <ee.featurecollection.FeatureCollection at 0x7f51be9d8190>),
 ('Nepal', <ee.featurecollection.FeatureCollection at 0x7f51be9d8ee0>),
 ('Pakistan', <ee.featurecollection.FeatureCollection at 0x7f51be9d1940>),
 ('India', <ee.featurecollection.FeatureCollection at 0x7f51be9cd640>),

### Step 4 - exporting 

In [18]:
for name, eedict in adjusted_area_data_sub_aois:
    name = name.replace(" ","_").replace("'","")
    fc = scripts.dashboard_data_to_fc(eedict)
    ee.batch.Export.table.toDrive(
        collection=fc,
        description=name,
        folder='test',
        fileFormat='csv',
        selectors='category,name,type,value'
    ).start()


In [9]:
#ee.batch.Export.image.toAsset(
#    image = output,
#    description = 'rap-aoi-c-elevation3500',
#    assetId = 'projects/ee-amcmahon/assets/seplan-test_recipe_2022-09-06_c-elevation3500',
#    region = aoi.geometry(),
#    scale = 30,
#    maxPixels = 1e13
#).start()